# Installs and Imports

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install lxml

install: missing destination file operand after 'requests'
Try 'install --help' for more information.


In [2]:
from bs4 import BeautifulSoup as bts
import lxml
import pandas as pd
import requests

# Defining Helper Functions

In [224]:
def get_and_parse_URL(url):
  html_text = requests.get(url, headers = {'User-Agent':'Mozilla/5.0'})
  soup = bts(html_text.text, 'lxml')
  return soup

def mix_same_columns(dataframe,column1,column2):
  for i in range(len(dataframe[column1])):
    if str(dataframe[column1].iloc[i]) == 'nan' and str(dataframe[column2].iloc[i]) != 'nan':
      dataframe[column1].iloc[i] = dataframe[column2].iloc[i]
    elif str(dataframe[column1].iloc[i]) != 'nan' and str(dataframe[column2].iloc[i]) == 'nan':
      dataframe[column1].iloc[i] = dataframe[column1].iloc[i]
  dataframe.drop(column2, inplace=True, axis=1)

def drop_columns(dataframe,list_of_columns):
  for column in list_of_columns:
    dataframe.drop(column, inplace=True, axis=1)

def zero_or_one(dataframe, column_name, value_for_one):
  for i in range(len(dataframe[column_name])):
    if  dataframe[column_name].iloc[i] == value_for_one:
      dataframe[column_name].iloc[i] = 1
    else:
      dataframe[column_name].iloc[i] = 0

def unique_values_for_columns(dataframe):
  list_of_column = ['Dahili Hafıza',
        'Ekran Çözünürlüğü', 'Ekran Cinsi', 'Ekran Boyutu Aralığı',
        'Arka Kamera Sayısı', 'Ön Kamera Çözünürlük Aralığı', 'Ön Kamera Flaş',
        'Renk', 'RAM Kapasitesi', 'Batarya Kapasitesi Aralığı',
        'Şarj Girişi', 'Kablosuz Şarj',
        'NFC', 'Parmak İzi Okuyucu', 'Yüz Tanıma', 'Çift Hat',
        'Ana Kamera Çözünürlül Aralığı' ]
  for column in list_of_column:
    print(column)
    print(dataframe[column].unique())
    
def sum_of_all_nulls(dataframe):
    columns = list(dataframe.columns)
    sum_of_nulls = 0
    for column in columns:
        sum_of_nulls += dataframe[column].isnull().sum()
    print('Sum of null values: '+str(sum_of_nulls))
    
def nulls_with_columns(dataframe):
    num = 0
    for column in list(dataframe.columns):
        if dataframe[column].isnull().any():
            print(column+': '+str(dataframe[column].isnull().sum()))
            num += 1
    if num != 0:
        sum_of_all_nulls(dataframe)
    else:
        print('There is no column with a null value.')
        
def columns_with_meaningless_value(dataframe):
    meaningless_value = ['Yok', 'Diğer']
    for column in list(dataframe.columns):
        if dataframe[column].isin(meaningless_value).any():
            if (dataframe[column].str.contains(meaningless_value[0])).any():
                print(column+': '+str(dataframe[column].isin(meaningless_value).sum())+f" rows and meaningless value is '{meaningless_value[0]}'.")
            else:
                print(column+': '+str(dataframe[column].isin(meaningless_value).sum())+f" rows and meaningless value is '{meaningless_value[1]}'.")
        
def manually_entering_meaningless_value(dataframe,column,meaningless_value,list_of_meaningful_value):
    for num,index in enumerate(dataframe[dataframe[column] == meaningless_value].index):
        dataframe[column].iloc[index] = list_of_meaningful_value[num]

# Web Scraping

***Step 1: Creating a list of total product URLs.***

In [ ]:
home_page_path = 'https://www.trendyol.com/sr?wb=794%2C101939%2C103505%2C109251%2C103503%2C109298%2C108166%2C142700%2C791%2C103502%2C145557%2C110936%2C108893%2C110476%2C110929%2C102323%2C103501%2C103504&wc=109460&attr=28%7C256&pi='
page_path_list = []
for i in range(1,32):
  page_path_list.append(home_page_path+str(i))

In [ ]:
product_path_list = []

for path in page_path_list:
  page = get_and_parse_URL(path)
  for div in page.find_all('div',{'class':'p-card-wrppr'}):
    product_path_list.append(div.a['href'])

***Step 2: Creating an empty dataframe with columns of headers.***

In [ ]:
columns = ['Brand Name','Price']
product = get_and_parse_URL('https://www.trendyol.com'+ product_path_list[0])
for li in product.find_all('li', {'class':'detail-attr-item'}):
  columns.append(li.find('span').text.strip())

In [ ]:
df_of_products = pd.DataFrame(columns=columns)

***Step 3: Pull data on a per product basis and add it to the dataframe.***

In [ ]:
for product_path in product_path_list:
  headers = ['Brand Name','Price','Product Name']
  values = []
  product = get_and_parse_URL('https://www.trendyol.com'+ product_path)

  brand_name = product.find('h1',{'class':'pr-new-br'}).a
  if brand_name != None:
    values.append(brand_name.text.strip())
  else:
    values.append(None)
  price = product.find('span',{'class':"prc-slg prc-slg-w-dsc"})
  if price != None:
    values.append(price.text.strip())
  else:
    price = product.find('span',{'class':"prc-slg"})
    values.append(price.text.strip())
  product_name = product.find('h1',{'class':'pr-new-br'}).span.text.strip()
  values.append(product_name)

  for li in product.find_all('li', {'class':'detail-attr-item'}):
    headers.append(li.find('span').text.strip())
    values.append(li.b.text.strip())
  data_series = pd.Series(values, index = headers)
  df_of_products = df_of_products.append(data_series, ignore_index=True)

***Step 4: Saving the dataframe as a csv file.***

In [21]:
df_of_products.to_csv('trendyol_data.csv', index = False)

# Data Cleaning

In [3]:
df = pd.read_csv('trendyol_data.csv')

In [4]:
df.head()

,Unnamed: 0,Brand Name,Price,Dahili Hafıza,İşlemci Markası,Garanti Tipi,Bağlantılar,Pil Gücü (mAh),Ekran Boyutu,Kamera Çözünürlüğü,...,Yüz Tanıma,Görüntülü Konuşma,Suya/Toza Dayanıklılık,Çift Hat,Ekran Yenileme Hızı,Video Kayıt Çözünürlüğü,Ana Kamera Çözünürlül Aralığı,Arttırılabilir Hafıza (Hafıza Kartı Desteği),Product Name,Cep Telefonu Modeli
0,0,Samsung,4.905 TL,128 GB,Diğer,Samsung TR Garantili,Wi-Fi,4000 - 5000,6 inç ve üstü,20 MP ve üstü,...,Var,Var,Var,Var,90 Hz,2160p (Ultra HD) 4K,60 - 80 MP,Var,Galaxy A52 128GB Siyah Cep Telefonu (Samsung T...,NaN
1,1,Xiaomi,4.019 TL,128 GB,NaN,Xiaomi Türkiye Garantili,NaN,NaN,NaN,NaN,...,Var,NaN,Var,Var,NaN,NaN,60 - 80 MP,Var,Redmi Note 10S 6GB + 128GB Siyah Cep Telefonu ...,NaN
2,2,POCO,5.199 TL,256 GB,Qualcomm,Xiaomi Türkiye Garantili,Wi-Fi,3000 - 7900,6 inç ve üstü,20 MP ve üstü,...,Var,Var,Var,Yok,120 Hz,2160p (Ultra HD) 4K,40 - 60 MP,Var,X3 Pro 8GB + 256GB Siyah Cep Telefonu (Xiaomi ...,NaN
3,3,Samsung,2.799 TL,64 GB,Samsung,Resmi Distribütör Garantili,Wi-Fi,4000 - 5000,6 inç ve üstü,20 MP ve üstü,...,Var,Var,Var,Var,90 Hz,1080p (Full HD),40 - 60 MP,Var,Galaxy M12 64GB Mavi Cep Telefonu (Samsung Tür...,NaN
4,4,Oppo,2.978 TL,128 GB,NaN,Resmi Distribütör Garantili,Wi-Fi,4000 - 5000,6 inç ve üstü,10 - 15 MP,...,Var,Var,Var,Yok,60 Hz,1080p (Full HD),10 - 15 MP,Var,A54 128GB Siyah Cep Telefonu (Oppo Türkiye Gar...,NaN


In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 48 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   Unnamed: 0                                    731 non-null    int64 
 1   Brand Name                                    731 non-null    object
 2   Price                                         731 non-null    object
 3   Dahili Hafıza                                 726 non-null    object
 4   İşlemci Markası                               508 non-null    object
 5   Garanti Tipi                                  731 non-null    object
 6   Bağlantılar                                   405 non-null    object
 7   Pil Gücü (mAh)                                527 non-null    object
 8   Ekran Boyutu                                  515 non-null    object
 9   Kamera Çözünürlüğü                            514 non-null    object
 10  Ek

***Step 1: Dropping unnecessary columns from the dataframe.***

In [261]:
columns_for_drop = ['Unnamed: 0','İşlemci Markası','Bağlantılar','Ekran Boyutu','Dokunmatik Ekran','Ana Kamera Flaş','İşletim Sistemi','Cep Telefonu Modeli','Ön Kamera Sayısı','Kamera Çözünürlüğü',
                    'Kulaklık Girişi','Bluetooth','GPS','Radio','Görüntülü Konuşma','Suya/Toza Dayanıklılık','Arttırılabilir Hafıza (Hafıza Kartı Desteği)','Video Kayıt Çözünürlüğü',
                    'Ekran Yenileme Hızı','Garanti Tipi','Mobil Bağlantı Hızı','Pil Gücü (mAh)']
drop_columns(df, columns_for_drop)

***Step 2: Sum all nulls and check for unique values per column.***

In [262]:
nulls_with_columns(df)

Dahili Hafıza: 5
Ekran Çözünürlüğü: 232
Görüntü Teknolojisi: 258
Ön Kamera Çözünürlüğü: 215
Ekran Boyutu Aralığı: 232
Ön Kamera Flaş: 271
CPU Aralık: 236
Şarj Girişi: 1
Şarj Hızı: 377
Kablosuz Şarj: 215
Parmak İzi Okuyucu: 206
Yüz Tanıma: 214
Ana Kamera Çözünürlül Aralığı: 2
Sum of null values: 2464


In [263]:
unique_values_for_columns(df)

Dahili Hafıza
['128 GB' '256 GB' '64 GB' '32 GB' nan '16 GB' '8 GB' '4 GB' 'Yok'
 '512 GB']
Ekran Çözünürlüğü
['FHD+' nan 'HD+' 'HD' 'QHD+' 'QHD' 'UHD+']
Ekran Cinsi
['Super AMOLED' 'AMOLED' 'LCD' 'PLS' 'Diğer' 'OLED']
Ekran Boyutu Aralığı
['6"-6.5"' nan '6.5" - 7"' '5"-5.5"' '5.5"-6"' '4.5"-5"' '7" üstü']
Arka Kamera Sayısı
['5' '4' '1' '3' 'Yok' '2' '10+']
Ön Kamera Çözünürlük Aralığı
['20 - 40 MP' '10 - 15 MP' '5 - 10 MP' '15 - 20 MP' '60 - 80 MP'
 '5 MP ve altı' 'Yok' '40 - 60 MP']
Ön Kamera Flaş
['Var' nan 'Yok']
Renk
['Siyah' 'Gri' 'Mavi' 'Yeşil' 'Beyaz' 'Mor' 'Metalik' 'Gümüş' 'Sarı'
 'Pembe' 'Altın' 'Lacivert' 'Çok Renkli' 'Turkuaz' 'Kırmızı' 'Bej'
 'Turuncu' 'Kahverengi' 'Haki']
RAM Kapasitesi
['8 GB' '6 GB' '4 GB' '3 GB' '2 GB' '0-1GB' '16 MB RAM' '1 GB' 'Yok'
 '12 GB' '1,5 GB RAM' '32 GB' '512 MB RAM']
Batarya Kapasitesi Aralığı
['4000-4500 mAh' '5000-6000 mAh' '4500-5000 mAh' '3500-4000 mAh'
 '1000-2000 mAh' '800-1000 mAh' '3000-3500 mAh' '2000-3000 mAh'
 '6000-7000 mAh']
Ş

***Step 3: Replacing binary options with 0's and 1's.***

In [264]:
zero_or_one(df,'Yüz Tanıma','Var')
zero_or_one(df,'Çift Hat','Var')
zero_or_one(df,'NFC','Var')
zero_or_one(df,'Ön Kamera Flaş','Var')
zero_or_one(df,'Kablosuz Şarj','Var')
zero_or_one(df,'Parmak İzi Okuyucu','Var')

***Step 4: Combining similar columns.***

In [265]:
mix_same_columns(df,'Ekran Çözünürlüğü','Ekran Teknolojisi')
mix_same_columns(df,'Ekran Boyutu Aralığı', 'Ekran Boyut Aralığı')
mix_same_columns(df,'Ekran Cinsi','Görüntü Teknolojisi')
mix_same_columns(df,'Ön Kamera Çözünürlük Aralığı','Ön Kamera Çözünürlüğü')

***Step 5: Deleting columns with a high value of null and columns with a high number of meaningless data  from our dataframe even though they are important.***

In [266]:
nulls_with_columns(df)

Dahili Hafıza: 5
CPU Aralık: 236
Şarj Girişi: 1
Şarj Hızı: 377
Ana Kamera Çözünürlül Aralığı: 2
Sum of null values: 621


In [267]:
columns_with_meaningless_value(df)

Dahili Hafıza: 16 rows and meaningless value is 'Yok'.
Ekran Çözünürlüğü: 80 rows and meaningless value is 'Diğer'.
Ekran Cinsi: 97 rows and meaningless value is 'Diğer'.
Arka Kamera Sayısı: 17 rows and meaningless value is 'Yok'.
Ön Kamera Çözünürlük Aralığı: 14 rows and meaningless value is 'Yok'.
RAM Kapasitesi: 9 rows and meaningless value is 'Yok'.
Şarj Girişi: 51 rows and meaningless value is 'Diğer'.


In [268]:
few_columns = ['CPU Aralık','Şarj Hızı','Ekran Çözünürlüğü','Ekran Cinsi','Şarj Girişi']
drop_columns(df, few_columns)

***Step 6: Manually entering data in columns with less Null values.***

In [269]:
nulls_with_columns(df)

Dahili Hafıza: 5
Ana Kamera Çözünürlül Aralığı: 2
Sum of null values: 7


In [270]:
df[df['Dahili Hafıza'].isnull()]['Product Name']

42     11T 8GB + 256GB Gri Cep Telefonu (Xiaomi Türki...
180    P40 Lite 128GB Yeşil Cep Telefonu (Huawei Türk...
296    P40 Lite 128GB Siyah Cep Telefonu (Huawei Türk...
342    11 Lite 5G NE 8GB + 128GB Beyaz Cep Telefonu (...
696          K10 16 Gb Siyah Cep Telefonu TYC00262159620
Name: Product Name, dtype: object

In [271]:
df['Dahili Hafıza'].iloc[42] = '256 GB'
df['Dahili Hafıza'].iloc[180] = '128 GB'
df['Dahili Hafıza'].iloc[296] = '128 GB'
df['Dahili Hafıza'].iloc[342] = '128 GB'
df['Dahili Hafıza'].iloc[696] = '16 GB'

In [272]:
df[df['Ana Kamera Çözünürlül Aralığı'].isnull()]['Product Name']

79     Galaxy S20 FE (Çift SIM) 128GB Cloud Lavender ...
722                                       Y6 P 2020 0035
Name: Product Name, dtype: object

In [273]:
df['Ana Kamera Çözünürlül Aralığı'].iloc[79] = '10 - 15 MP'     
df['Ana Kamera Çözünürlül Aralığı'].iloc[722] = '10 - 15 MP'     

In [274]:
nulls_with_columns(df)

There is no column with a null value.


***Step 7: Making meaningless value in columns meaningful.***

In [275]:
columns_with_meaningless_value(df)

Dahili Hafıza: 16 rows and meaningless value is 'Yok'.
Arka Kamera Sayısı: 17 rows and meaningless value is 'Yok'.
Ön Kamera Çözünürlük Aralığı: 14 rows and meaningless value is 'Yok'.
RAM Kapasitesi: 9 rows and meaningless value is 'Yok'.


In [276]:
dahili_hafiza_list = ['16 GB','16 GB','256 GB','32 GB','64 GB','16 GB','16 GB','64 GB','64 GB','32 GB','64 GB','16 GB','16 GB','16 GB','64 GB','16 GB' ]
manually_entering_meaningless_value(df,'Dahili Hafıza','Yok',dahili_hafiza_list)

In [ ]:
arka_kamera_list = []
manually_entering_meaningless_value(df,'Arka Kamera Sayısı','Yok',arka_kamera_list)

In [ ]:
on_kamera_list = []
manually_entering_meaningless_value(df,'Ön Kamera Çözünürlük Aralığı','Yok',on_kamera_list)

In [ ]:
ram_list = []
manually_entering_meaningless_value(df,'RAM Kapasitesi','Yok',ram_list)